In [1]:
from engine import Value
from mlp import MLP, Layer, Neuron
import numpy as np
import random

In [2]:

# 5X5 image
image = [[4,5,4,4,4],
         [4,5,4,4,4],
         [4,5,4,4,4],
         [4,5,4,4,4],
         [4,5,4,4,4]]

print(image)

# image = np.random.randn(5,5,1)

# print(image, image.shape)

[[4, 5, 4, 4, 4], [4, 5, 4, 4, 4], [4, 5, 4, 4, 4], [4, 5, 4, 4, 4], [4, 5, 4, 4, 4]]


In [3]:
# 2X2 filter
filter = [[3,1],
          [4,5]]

print(filter)

# filter = np.random.randn(2,2)

# print(filter, filter.shape)

[[3, 1], [4, 5]]


In [4]:
# # conv operation

# def conv(image, filter):
#     final_image_size = int(image.shape[0] - filter.shape[0] + 1)
#     # print(final_image_size, type(final_image_size))
#     final_image = np.zeros([final_image_size, final_image_size])
#     final_image = [[0 for x in range(fina)] for y in range(final_image_size)] 
    
#     for i in range(final_image_size):
#         # print(i)
#         for j in range(final_image_size):
#             # Apply the filter to the current position
#             for x in range(filter.shape[0]):
#                 for y in range(filter.shape[1]):
#                     # print(final_image)
#                     if i+x < image.shape[0] and j+y < image.shape[1]:
#                         final_image[i][j] += image[i+x][j+y] * filter[x][y]
            
#     return final_image

# conv(image, filter)

In [5]:
import time

def flatten(image, image_size):
    height = image_size
    width = image_size

    flat_array = []

    flat_array = [0]*(height*width)
    cnt = 0

    for i in range(height):
        for j in range(width):
            flat_array[cnt] = image[i][j]
            cnt += 1

    return flat_array

flatten(image, 5)

[4, 5, 4, 4, 4, 4, 5, 4, 4, 4, 4, 5, 4, 4, 4, 4, 5, 4, 4, 4, 4, 5, 4, 4, 4]

In [6]:
def mse_loss(ypred, y_train):
    return (ypred - y_train)**2

In [7]:
# input_image = np.array([[4,5,4,4,4],
#          [4,5,4,4,4],
#          [4,5,4,4,4],
#          [4,5,4,4,4],
#          [4,5,4,4,4]])

# true_label = 1.0


# filter = [[Value(0.5),Value(0.5)],
#           [Value(0.5),Value(0.5)]]

# fc_model = MLP(16, [16, 16, 1])

# for i in range(3):

#     convoluted_image = conv(input_image, 5, filter, 2)
#     flattened_image = flatten(convoluted_image, 4)
    
#     outputs = fc_model(flattened_image)

#     print(filter, "\n")

#     loss = mse_loss(outputs, true_label)

#     # zero_grad
#     for p in fc_model.parameters():
#         p.grad = 0.0

#     loss.backward()

#     # update
#     for p in fc_model.parameters():
#         p.data += -0.1 * p.grad

#     print("loss:", loss, "\n")

In [8]:
class Conv2d:
    def __init__(self, image_size, filter_size, pool='None'):
        self.image_size = image_size
        self.filter_size = filter_size
        self.filter = [[Value(5.0, label='conv') for _ in range(filter_size)] for _ in range(filter_size)]
        self.pool = pool

    def __call__(self, image):
        final_image_size = int(self.image_size - self.filter_size + 1)
        # print(final_image_size, type(final_image_size))
        final_image = np.zeros([final_image_size, final_image_size])
        final_image = [[Value(0.0) for x in range(final_image_size)] for y in range(final_image_size)] 
        
        for i in range(final_image_size):
            for j in range(final_image_size):
                
                for x in range(self.filter_size):
                    for y in range(self.filter_size):

                        if i+x < self.image_size and j+y < self.image_size:
                            final_image[i][j] += (image[i+x][j+y] * self.filter[x][y])
        print(final_image)
        if self.pool == 'Max':
            pooled_image = [[Value(0.0) for x in range(final_image_size//2)] for y in range(final_image_size//2)] 
            for i in range(final_image_size//2):
                for j in range(final_image_size//2):
                    max_num = -1e9
                    for x in range(2):
                        for y in range(2):

                            if i+x < self.image_size and j+y < self.image_size:
                                pooled_image[i][j] = max(max_num, final_image[i][j])
        

        return final_image
    
    def parameters(self):
        # print(self.filter)
        o = flatten(self.filter, self.filter_size)
        # all_grads = [x.grad for x in o]
        # print(all_grads)
        return o

# conv(image, 5, filter, 2)

In [9]:
import torch.nn as nn
import torch.nn.functional as F

# net = Net()

class CNN:
    def __init__(self):
        self.conv1 = Conv2d(5, 2)
        self.fc = MLP(2, [6,6,1])
    
    def __call__(self, input_image):
        outputs = self.conv1(input_image)
        outputs = flatten(outputs, 4)
        
        outputs = self.fc(outputs)
        return outputs
    
    def parameters(self):
        # print(self.conv1.parameters())
        return self.fc.parameters() + self.conv1.parameters()

In [17]:
image = [[4,5,4,4,4],
         [4,5,4,4,4],
         [4,5,4,4,4],
         [4,5,4,4,4],
         [4,5,4,4,4]]

label = 1.0

cnn_model = CNN()

for i in range(5):
    
    #forward
    outputs = cnn_model(image)
    loss = mse_loss(outputs, label)

    # zero_grad
    for p in cnn_model.parameters():
        p.grad = 0.0

    loss.backward()

    # update
    for p in cnn_model.parameters():
        p.data += -0.1 * p.grad

    print(f"Epoch {i}, Loss: {loss.data}")

Epoch 0, Loss: 3.948370242039453
Epoch 1, Loss: 3.940797887082048
Epoch 2, Loss: 3.930759833880115
Epoch 3, Loss: 3.9168857977113136
Epoch 4, Loss: 3.8966200913655125
